In [1]:
# api_key = "y6D1BHRcjTDCcaOi21C3LuwHpYxe0itpoOYca1DTYDlFNrNBi5OZkeFGK4rBb4Ga"
# api_secret = "c4pIQ5H5s9d1ezmTrfxAfguBTC4f6QFLoBEK4ftokm9yEPBbVyqDF05dxCbTnr57"

In [2]:
# client = Client(api_key=api_key, api_secret=api_secret, tld = "com")

In [3]:
# client.get_symbol_info('BTCUSDT')

In [4]:
# pd.set_option('display.max_rows', None)

In [5]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import binance as bn
import pandas as pd
import sys
sys.path.append('/Users/jp/Desktop/Investment/utils')
import utils as utils
from datetime import datetime, timedelta
import ta as ta
import numpy as np

In [6]:
class Long_macd_simp_trader():
    
    """Class to perform live testing using Binance testnet stream of data
    """
    def __init__(self, symbol=None, units='0.0006', interval=None, ema_slow=None, ema_fast=None, ema_signal=None, sesion_end=None, testnet=None, duration_minutes=None):
        
        """Long_macd_trader class constructor
        :param interval: a string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :position: initial position in the market. 1 for long (have the asset) 0 for neutral (dont have the asset)
        :position: integer
        :back_testing: set to True if backtesting is necessary
        """
        self.units = units
        self.symbol = symbol
        self.position = None #initially no position is decided, it is pending to analysis recent data of macd to decide
                             #what is the best way to enter the market
        self.interval = interval
        self.ema_slow = ema_slow
        self.ema_fast = ema_fast
        self.ema_signal = ema_signal 
#         self.start = current time to be assigned, format to be decided
#         self.sesion_end = time at which we want the trading sesion to stop automatically
        self.testnet = testnet #if the sesion takes place in the testnet
        self.data = pd.DataFrame()
        self.tct = None #turned complete time
        self.client = None 
        self.trades = 0 #counter of the number of trades
        self.trade_values = [] #all trading positions consecutive, long/short... or short/long...
        self.duration_minutes = duration_minutes # duration of the trading sesion in minutes
        self.trade_start_time_utc = None # time in utc to be defined when the stream of OHLC starts ( this time
        #is used to extract the difference between actual time of trading sesion in utc time and the index of the
        # last kandle data that it is given also in utc time by binance api)
    
    def stream_candles(self, msg):
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time, 'Open'] = first
        self.data.loc[start_time, 'High'] = high
        self.data.loc[start_time, 'Low'] = low
        self.data.loc[start_time, 'Close'] = close
        self.data.loc[start_time, 'Volume'] = volume
        self.data.loc[start_time, 'Complete'] = complete

        #update MACD parameters with each ws retrieval
        macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
        macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
        macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()         
        self.data.loc[start_time, 'macd_diff'] = macd_diff.iloc[-1]
        self.data.loc[start_time, 'macd_macd'] = macd_macd.iloc[-1]
        self.data.loc[start_time, 'macd_signal'] = macd_signal.iloc[-1]  
        #check start_time coming from ws matches with the data in the last index of the data dataframe
        # print out
        print(f"l_t: {event_time}|st: {start_time}|l_p: {format(self.data.Close[-1],'.2f')}|macdd: {format(self.data.macd_diff[-1],'.2f')}|p: {self.position} |c: {complete}")      
        
        # print out
#         print(".", end = "", flush = True) # just print something to get a feedback (everything OK)
        dt = self.data.index[-1] - self.trade_start_time_utc
        if ((dt) > timedelta(minutes=self.duration_minutes)):
            self.stop_ses()
            print(f"trading sesion duration = {dt.seconds/60} minutes")
            if self.position != 0:
                order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP") 
                self.position = 0
            else: 
                print("STOP")
        
        if (complete == True):
            self.tct = start_time #time of the lastest updated/incorporated kandle            
            self.define_strategy()
            self.execute_trades()
                
    def start_trading(self):
        self.trade_start_time_utc = datetime.utcnow()
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        self.prepare_recent_data()
        self.twm.start_kline_socket(callback = self.stream_candles, symbol = self.symbol, interval = self.interval)
        
    def stop_ses(self):
        self.twm.stop()
    
    def define_strategy(self):
        macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
        macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
        macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()
        #save index of latest position in this cycle which is equivalent to the position in last read "start_time",
        #which is the latest bar that has just turned into a complete one
#         print('self.tct', self.tct)
#         print('macd_diff.index', macd_diff.index)
#         print('macd_diff', macd_diff.iloc[-1])
#         print(self.data.loc[self.tct,'macd_diff'])
        cond_index = self.data.index == self.tct
        self.data.loc[cond_index, 'macd_diff'] = macd_diff.loc[self.tct]
        self.data.loc[cond_index, 'macd_macd'] = macd_macd.loc[self.tct]
        self.data.loc[cond_index, 'macd_signal'] = macd_signal.loc[self.tct]
        #stablish neutral conditions taking into account the latest complete kandle and the previous one
        ht_previous = self.data.macd_diff.iloc[-2]# this is the previous kandle to the just complete one
        ht_last = self.data.macd_diff.iloc[-1] #this is the last kandle that has just turned into a complete one
        if ((ht_previous > 0) and (ht_last < 0)):
            self.position = 0 #position adopted in the latest kandle that has turned into complete
        elif ((ht_previous < 0) and (ht_last > 0)):
            self.position = 1 #position adopted in the latest kandle that has turned into complete
        else:  
            self.position = None #in any other case do nothing and continue

    def execute_trades(self):
        #execute trades depending on the positions stablished in define_strategy()
            if self.position == 1:
                order = self.client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  
            if self.position == 0:
                order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")
            if self.position == None:
                pass
        
    def report_trade(self, order, going):
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")
        
    def prepare_recent_data(self):
        '''
        REMARK: Introduced time must be in Tokyo time (UTC+9) but the calculations will be in UTC
        Prepare all the fields of dat a necessary for the study. The interval of dates to be studied is the one
        given when delclaring the class. To prepare another interval of dates, please create another class instance.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :param interval: string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        '''
        
        #Prepare pre-data to the present moment with non NaN for the trading period (macd paramters ready)
        current_time_obj = datetime.now()
        current_time = int(current_time_obj.timestamp()*1000)

#         from_time_obj = datetime.strptime(start, "%Y-%m-%d-%H:%M")
#       ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        ema_diff = self.ema_slow + self.ema_signal + 2
        td = timedelta()
        if 'm' in self.interval:
            num_min = int(self.interval.replace('m',''))
            td = timedelta(minutes=num_min*(ema_diff))
        if 'h' in self.interval:
            num_h = int(self.interval.replace('h',''))
            td = timedelta(hours=num_h*(ema_diff))
        if 'd' in self.interval:
            num_day = int(self.interval.replace('d',''))
            td = timedelta(days=num_day*(ema_diff))
        if 'w' in self.interval:
            num_week = int(self.interval.replace('w',''))
            td = timedelta(weeks=num_week*(ema_diff))
        if 'M' in self.interval:
            num_week_m = int(self.interval.replace('M',''))
            td = timedelta(weeks=num_week_m * 4 * (ema_diff))
        from_time_obj = current_time_obj - td
        from_time = int((current_time_obj - td).timestamp()*1000)
        self.data, self.client = utils.get_history_v3(symbol=self.symbol, interval=self.interval, start=from_time, end=current_time, testnet=self.testnet)
        #obtaining MACD instance from python ta
        macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
        macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
        macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()
        #assigning the values of macd to ticker dataframe
        self.data['macd_diff'] = macd_diff
        self.data['macd_macd'] = macd_macd
        self.data['macd_signal'] = macd_signal
        #last candle retrieved from old data till the present moment will be incomplete 99.9% of the time
        self.data["Complete"] = [True for row in range(len(self.data)-1)] + [False]
        #decide position to be given to the latest recent data kandle (not completed in 99% of cases and that is 
        #going to be updated by the stream of data, so actually, position given to the latest kandle at this point
        #it is mots likely not to be used, but updated.
        ht_previous = self.data.macd_diff.iloc[-2]# this is the kandle before the latest (complete in recent data)
        ht_last = self.data.macd_diff.iloc[-1] #this is the last (incomplete 99% cases)
        if ((ht_previous > 0) and (ht_last < 0)):
            self.position = 0 #position adopted in the latest kandle that is incomplete 99% cases
        elif ((ht_previous < 0) and (ht_last > 0)):
            self.position = 1 #position adopted in the latest kandle that is incomplete 99% cases
        else:  
            self.position = None #in any other case do nothing and continue

In [7]:
trader_test = Long_macd_simp_trader(symbol="BTCUSDT", units='0.0006',interval='1m', ema_slow=26, ema_fast=12, ema_signal=9, testnet=True, duration_minutes=10)

In [14]:
trader_test.start_trading()

37
l_t: 2022-10-02 15:29:12.370000|st: 2022-10-02 15:29:00|l_p: 19116.88|macdd: -2.06|p: None |c: False
l_t: 2022-10-02 15:29:14.430000|st: 2022-10-02 15:29:00|l_p: 19117.89|macdd: -2.00|p: None |c: False
l_t: 2022-10-02 15:29:16.736000|st: 2022-10-02 15:29:00|l_p: 19117.88|macdd: -2.00|p: None |c: False
l_t: 2022-10-02 15:29:19.054000|st: 2022-10-02 15:29:00|l_p: 19117.77|macdd: -2.01|p: None |c: False
l_t: 2022-10-02 15:29:21.064000|st: 2022-10-02 15:29:00|l_p: 19118.35|macdd: -1.97|p: None |c: False
l_t: 2022-10-02 15:29:23.117000|st: 2022-10-02 15:29:00|l_p: 19112.56|macdd: -2.34|p: None |c: False
l_t: 2022-10-02 15:29:25.164000|st: 2022-10-02 15:29:00|l_p: 19112.66|macdd: -2.33|p: None |c: False
l_t: 2022-10-02 15:29:27.269000|st: 2022-10-02 15:29:00|l_p: 19107.98|macdd: -2.63|p: None |c: False
l_t: 2022-10-02 15:29:29.297000|st: 2022-10-02 15:29:00|l_p: 19107.60|macdd: -2.66|p: None |c: False
l_t: 2022-10-02 15:29:31.325000|st: 2022-10-02 15:29:00|l_p: 19106.82|macdd: -2.71|p: No

l_t: 2022-10-02 15:32:02.058000|st: 2022-10-02 15:32:00|l_p: 19124.65|macdd: -0.42|p: None |c: False
l_t: 2022-10-02 15:32:04.121000|st: 2022-10-02 15:32:00|l_p: 19123.55|macdd: -0.49|p: None |c: False
l_t: 2022-10-02 15:32:06.187000|st: 2022-10-02 15:32:00|l_p: 19123.91|macdd: -0.46|p: None |c: False
l_t: 2022-10-02 15:32:08.204000|st: 2022-10-02 15:32:00|l_p: 19123.91|macdd: -0.46|p: None |c: False
l_t: 2022-10-02 15:32:10.243000|st: 2022-10-02 15:32:00|l_p: 19123.46|macdd: -0.49|p: None |c: False
l_t: 2022-10-02 15:32:12.260000|st: 2022-10-02 15:32:00|l_p: 19123.89|macdd: -0.46|p: None |c: False
l_t: 2022-10-02 15:32:14.358000|st: 2022-10-02 15:32:00|l_p: 19124.11|macdd: -0.45|p: None |c: False
l_t: 2022-10-02 15:32:16.371000|st: 2022-10-02 15:32:00|l_p: 19124.36|macdd: -0.43|p: None |c: False
l_t: 2022-10-02 15:32:18.465000|st: 2022-10-02 15:32:00|l_p: 19124.64|macdd: -0.42|p: None |c: False
l_t: 2022-10-02 15:32:20.494000|st: 2022-10-02 15:32:00|l_p: 19123.78|macdd: -0.47|p: None 

l_t: 2022-10-02 15:34:45.741000|st: 2022-10-02 15:34:00|l_p: 19126.96|macdd: 0.27|p: 1 |c: False
l_t: 2022-10-02 15:34:47.826000|st: 2022-10-02 15:34:00|l_p: 19127.39|macdd: 0.30|p: 1 |c: False
l_t: 2022-10-02 15:34:49.832000|st: 2022-10-02 15:34:00|l_p: 19127.24|macdd: 0.29|p: 1 |c: False
l_t: 2022-10-02 15:34:51.833000|st: 2022-10-02 15:34:00|l_p: 19127.29|macdd: 0.29|p: 1 |c: False
l_t: 2022-10-02 15:34:53.834000|st: 2022-10-02 15:34:00|l_p: 19129.46|macdd: 0.43|p: 1 |c: False
l_t: 2022-10-02 15:34:55.841000|st: 2022-10-02 15:34:00|l_p: 19132.30|macdd: 0.61|p: 1 |c: False
l_t: 2022-10-02 15:34:57.903000|st: 2022-10-02 15:34:00|l_p: 19133.57|macdd: 0.69|p: 1 |c: False
l_t: 2022-10-02 15:34:59.940000|st: 2022-10-02 15:34:00|l_p: 19138.24|macdd: 0.99|p: 1 |c: False
l_t: 2022-10-02 15:35:00.004000|st: 2022-10-02 15:34:00|l_p: 19137.97|macdd: 0.97|p: 1 |c: True
l_t: 2022-10-02 15:35:02.013000|st: 2022-10-02 15:35:00|l_p: 19138.05|macdd: 1.46|p: None |c: False
l_t: 2022-10-02 15:35:04.076

l_t: 2022-10-02 15:37:35.804000|st: 2022-10-02 15:37:00|l_p: 19128.42|macdd: 0.89|p: None |c: False
l_t: 2022-10-02 15:37:38.085000|st: 2022-10-02 15:37:00|l_p: 19127.41|macdd: 0.82|p: None |c: False
l_t: 2022-10-02 15:37:40.245000|st: 2022-10-02 15:37:00|l_p: 19127.01|macdd: 0.80|p: None |c: False
l_t: 2022-10-02 15:37:42.264000|st: 2022-10-02 15:37:00|l_p: 19129.97|macdd: 0.98|p: None |c: False
l_t: 2022-10-02 15:37:44.337000|st: 2022-10-02 15:37:00|l_p: 19129.47|macdd: 0.95|p: None |c: False
l_t: 2022-10-02 15:37:46.558000|st: 2022-10-02 15:37:00|l_p: 19129.34|macdd: 0.94|p: None |c: False
l_t: 2022-10-02 15:37:48.558000|st: 2022-10-02 15:37:00|l_p: 19130.59|macdd: 1.02|p: None |c: False
l_t: 2022-10-02 15:37:50.640000|st: 2022-10-02 15:37:00|l_p: 19129.60|macdd: 0.96|p: None |c: False
l_t: 2022-10-02 15:37:52.727000|st: 2022-10-02 15:37:00|l_p: 19129.50|macdd: 0.95|p: None |c: False
l_t: 2022-10-02 15:37:54.757000|st: 2022-10-02 15:37:00|l_p: 19129.34|macdd: 0.94|p: None |c: False


In [15]:
trader_test.stop_ses()